# Web Scrapping Comida Di Buteco 

Para facilitar nosso projeto, vamos criar um script que vai processar os dados do Comida di Boteco, Como um webScrapping Simulado.

Infelizmente o site sugerido na documentação com os detalhes dos pratos de cada boteco não está mais disponível, então vamos usar outro site como referencia.


## Extração dos dados: 

Para não termos que lidar com problemas de anti-scrapping, vamos usar uma forma manula de fazer scraping da página. Para isso faça os seguintes passos:


1. Abra o site [https://bhaz.com.br/guia-bhaz/comida-di-buteco-2025-lista-pratos-concorrentes-bh/](https://bhaz.com.br/guia-bhaz/comida-di-buteco-2025-lista-pratos-concorrentes-bh/) em seu navegador.
2. Abra o console do navegador (F12 ou Ctrl+Shift+J ou Cmd+Option+J).
3. Ao abrir o console, caso seja necessário, clique na aba "Console".
4. Copie o seguinte código e cole no console:
5. ```javascript
   copy(document.body.innerHTML);
    ```
6. Pressione Enter para executar o código. (Isso irá copiar todo o conteúdo HTML da página para a área de transferência).
7. Vá para um editor de texto (como o Notepad, Visual Studio Code, etc.) e cole o conteúdo copiado.
8. Salve o arquivo como `./data/comida_di_boteco.html` no diretório deste  projeto.
9. Agora você tem o HTML da página salvo localmente e pode usá-lo para processar os dados, veja em [comida_di_boteco.html](./data/comida_di_boteco.html) o conteúdo do arquivo.

In [1]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
import pandas as pd 
from unidecode import unidecode
import re


## Settings 

In [2]:
FILE_HTML = './data/comida_di_boteco.html'

## Read HTML file

In [ ]:
from bs4 import BeautifulSoup

with open(FILE_HTML, "r", encoding="utf-8") as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, "html.parser")

### Extracting Plates Information 

In [ ]:
h2_tag = soup.find("h2", id="h-pratos-do-comida-di-buteco-2025")

div_parent = h2_tag.find_parent("div")

print(div_parent.prettify())

<div class="elementor-widget-container" data-loaded="true">
 <p data-loaded="true">
  Um dos mais queridos concursos gastronômicos de Belo Horizonte entrou na reta final, mas não se preodupe: até 11 de maio, o Comida di Buteco 2025 reforça a paixão mineira por tira-gostos e pratos boêmios. Na tarefa de te ajudar a se organizar para aproveitar ao máximo estes últimos duas,
  <strong>
   o BHAZ preparou a lista de petiscos confirmados.
  </strong>
 </p>
 <div class="wp-block-buttons is-content-justification-center is-layout-flex wp-container-core-buttons-layout-1 wp-block-buttons-is-layout-flex" data-loaded="true">
  <div class="wp-block-button">
   <a class="wp-block-button__link has-white-color has-luminous-vivid-orange-background-color has-text-color has-background has-link-color wp-element-button" href="https://chat.whatsapp.com/Ld501y9SQUL74xKsr4Niut" rel="noreferrer noopener" target="_blank">
    <strong>
     Clique aqui e faça parte da comunidade Rolê BHAZ no WhatsApp
    </stron

In [ ]:
filtered_children = []
for child in div_parent.children:
    if child.name in ["h4", "ul"]:
        filtered_children.append(child)
    elif child.name == "div" and "wp-block-image" in child.get("class", []):
        filtered_children.append(child)
filtered_children[:3]

[<h4 class="wp-block-heading" data-loaded="true" id="h-alexandre-s-bar"><strong>Alexandre’s Bar</strong></h4>,
 <ul data-loaded="true"><li>O fabuloso discão: Polpetone com recheio de muçarela acompanhada com caponata de berinjela e abobrinha.</li><li>Endereço: Rua David Alves do Vale, 68 | Santa Rosa, Belo Horizonte</li></ul>,
 <div class="wp-block-image" data-loaded="true"><figure class="aligncenter is-resized"><img alt="O fabuloso discão do Alexandre's Bar — Foto: Isaque Rodrigues/Comida di Buteco" decoding="async" src="https://s2-g1.glbimg.com/w6xgTZb45ExVKcLoRAFIaPl2iwA=/0x0:3500x2333/984x0/smart/filters:strip_icc()/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2025/T/o/nXdbyMTvuav3ZpGEAotA/alexandresbar-fabulosodiscao-bh-2025-isaquerodrigues-unibh-8111.jpg" style="width:648px;height:auto"/><figcaption class="wp-element-caption">Isaque Rodrigues/Comida di Buteco</figcaption></figure></div>]

In [6]:
len(filtered_children),len(filtered_children)//3

(369, 123)

### Consolidating the Dataset

In [7]:
treated_data = []

for i in tqdm(range(0, len(filtered_children), 3)):
    h4_content = None
    plate_name = None
    plate_description = None
    endereco = None
    image_path = None
        
    chunk = filtered_children[i:i+3]
    if len(chunk) < 3:
        print("Incomplete chunk, skipping...")
        continue
    
    title = chunk[0]
    unordered_list = chunk[1]
    image_div = chunk[2]


    if title.name == "h4" and "wp-block-heading" in title.get("class", []):
        h4_content = title.get_text(strip=True)
    
    if unordered_list.name == "ul":
        li_items = unordered_list.find_all("li")
        if len(li_items) >= 2:
            first_li_content = li_items[0].get_text(strip=True)
            second_li_content = li_items[1].get_text(strip=True)

            if first_li_content.find(":")!= -1:
                plate_name = first_li_content.split(":")[0].strip()
                plate_description = first_li_content.split(":")[1].strip()
            
            if second_li_content.find(":")!= -1:
                endereco = second_li_content.split(":")[1].strip()

    if image_div.name == "div" and "wp-block-image" in image_div.get("class", []):
        img_tag = image_div.find("img")
        if img_tag and img_tag.has_attr("src"):
            image_path = img_tag["src"]


    if h4_content and plate_name and plate_description and endereco and image_path:
        treated_data.append({
            "bar_name": h4_content,
            "plate_name": plate_name,
            "plate_description": plate_description,
            "endereco": endereco,
            "image_path": image_path
        })


100%|██████████| 123/123 [00:00<00:00, 18040.33it/s]


In [8]:
treated_data[:10]

[{'bar_name': 'Alexandre’s Bar',
  'plate_name': 'O fabuloso discão',
  'plate_description': 'Polpetone com recheio de muçarela acompanhada com caponata de berinjela e abobrinha.',
  'endereco': 'Rua David Alves do Vale, 68 | Santa Rosa, Belo Horizonte',
  'image_path': 'https://s2-g1.glbimg.com/w6xgTZb45ExVKcLoRAFIaPl2iwA=/0x0:3500x2333/984x0/smart/filters:strip_icc()/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2025/T/o/nXdbyMTvuav3ZpGEAotA/alexandresbar-fabulosodiscao-bh-2025-isaquerodrigues-unibh-8111.jpg'},
 {'bar_name': 'Amarelim do Prado',
  'plate_name': 'Trem de roças',
  'plate_description': 'Bochecha de porco na cama de angu mole, pesto de horta, mandioca e agrião. Acompanha pão e patê de fígado de galinha.',
  'endereco': 'Avenida Francisco Sá, 658 | Prado, Belo Horizonte',
  'image_path': 'https://s2-g1.glbimg.com/A1sXFeWp6YE12bpInykstKTPpHc=/0x0:3500x2333/984x0/smart/filters:strip_icc()/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538

### Creating DataFrame 

In [9]:
df_comida = pd.DataFrame(treated_data, columns=[
    "bar_name", "plate_name", "plate_description", "endereco", "image_path"
])

In [10]:

renamed_bars = {
    'Espetinhos do Paulão': 'Espetinho do Paulão',
    'Arcos Bar': 'Arcos Bar e Restaurante',
    'Azougue Fogo e Bar': 'Azougue Parrilla',
    'Baiúca': 'Baiúca Picanha E Cia',
    'Bar e Restaurante do Joãozinho': 'Joãozinho Bar e Restaurante',
    'Bar Estabelecimento': 'Estabelecimento',
    'Bar Junto Juntinho': 'Junto Juntinho Bar',
    'Bar Mania Mineira': 'Mania Mineira',
    'Bar Temático Santa Tereza': 'Bar Temático de Santa Tereza',
    'Barrigudinha Buteco': 'Barrigudinha',
    'Beco Restaurante': 'Restaurante Beco',
    'Benjamin Bar': 'Benjamin Restaurante Bar',
    'Benjamin Bar': 'Benjamin Restaurante Bar',
    'Buteco Tô D’Boa': "Buteco Tô D' Boa", 
    'Camisola Bar': 'Camisola Bar e Restaurante',
    'Canela Amarela': 'Pé de Canela Amarela',
    'Chopp da Esquina': 'Chop da Esquina',
    'Chopp da Esquina': 'Chop da Esquina',
    'Dona Suica': 'Restaurante Dona Suica',
    'Espettinho.com': 'ESPETO.COM',
    'Iracema bar': 'Iracema Bistro bar',
    'Já To Inno': 'Churrascaria Já Tô Inno',
    'Juzé Bar':'Juzé Restaurante - Bar',
    'Lá Ele Bar': 'ACARAJE DA ALE E LÁ ELE BAR',
    'Locomotivas Bar': "Locomotiva's Bar",
    'Marina’s Bar': 'MARINAS BAR',
    'PoiZé Bar e Petisqueira': 'PoiZé Bar e Petisqueria',
}

df_comida['bar_name'] = df_comida['bar_name'].replace(renamed_bars)
df_comida


,bar_name,plate_name,plate_description,endereco,image_path
0,Alexandre’s Bar,O fabuloso discão,Polpetone com recheio de muçarela acompanhada ...,"Rua David Alves do Vale, 68 | Santa Rosa, Belo...",https://s2-g1.glbimg.com/w6xgTZb45ExVKcLoRAFIa...
1,Amarelim do Prado,Trem de roças,"Bochecha de porco na cama de angu mole, pesto ...","Avenida Francisco Sá, 658 | Prado, Belo Horizonte",https://s2-g1.glbimg.com/A1sXFeWp6YE12bpInykst...
2,Andrade’s Beer,Chora canelinha,"Ossobuco com purê de batatas, acompanhado de t...","Rua Dona Geni, 32 | Jardim Florência (Venda No...",https://s2-g1.glbimg.com/9tZo4O55GproBN850s0km...
3,Arcos Bar e Restaurante,Panela da Diversidade,"Acém cozido com ervas finas, acompanhado com b...","Rua da Bahia, 1144 | Centro, Belo Horizonte.",https://s2-g1.glbimg.com/TBJlO2gBA-eoaA_0XK6kM...
4,Armazém Santa Amélia,Segredo do Armazém,Polenta cremosa com ragu de carnes cheio de sa...,"Avenida da Sinfonia, 253 | Santa Amélia, Belo ...",https://s2-g1.glbimg.com/z23BBsunx09X3iGDgJq31...
...,...,...,...,...,...
116,Xambar,Mineirin Oriental,"Miolo de acém assado na pressão, acompanhado d...","Avenida Augusto de Lima, 233 | Centro, Belo Ho...",https://cdb-static-files.s3.amazonaws.com/wp-c...
117,Xico da Kafua,Pé na areia,"Mão suína, acompanhada de farofa crocante de s...","Avenida Itaú, 1195 – Loja A | João Pinheiro, B...",https://cdb-static-files.s3.amazonaws.com/wp-c...
118,Xico do Churrasco,Xico Balaio,Asinhas de frango com provolone com tempero.,"Av Altamiro avelino Soares 920 – Castelo, Belo...",https://cdb-static-files.s3.amazonaws.com/wp-c...
119,Zé Bolacha,Rabicó,Bolinho de costelinha desfiada com requeijão e...,"Rua das Melancias, 35 | Vila Cloris, Belo Hori...",https://comidadibuteco.com.br/wp-content/uploa...


### Saving the DataFrame to a CSV file

In [11]:
df_comida.to_csv('./data/comida_di_boteco_2025.csv', index=False)